<a href="https://colab.research.google.com/github/aleeepassarelli/scientific-validation-hub/blob/main/notebooks/validation_CCC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛡️ GENIA FRAMEWORK: Hybrid Contract Validator (HCV)

Esta ferramenta implementa a **Validação Híbrida** para Agentes de IA, superando as limitações da similaridade de cossenos pura.

### A Equação de Confiança
$$Score_{Final} = (Score_{Vetorial} \times W_v) + (Score_{Booleano} \times W_b)$$

Onde:
* **Score Vetorial ($W_v \approx 0.3$):** Mede a "vibe", o tom e a densidade semântica (Embeddings).
* **Score Booleano ($W_b \approx 0.7$):** Mede o cumprimento estrito de regras lógicas (Verificado por um LLM Juiz).

---
**Objetivo:** Validar se o Agente cumpriu o contrato: *"Seja seguro, técnico e sem piadas."*

In [ ]:
# @title 1. Instalação das Dependências (Sentence Transformers)
!pip install sentence-transformers numpy -q

import json
import numpy as np
from typing import Dict, List, Any
from sentence_transformers import SentenceTransformer, util

# Carregamento do modelo de Embeddings (Cacheado)
print("⏳ Carregando modelo de Embeddings (Soft Logic)...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Modelo carregado.")

[Code]: O Motor do Juiz LLM (Hard Logic)
Nota: Para este teste, criei um "Mock" simulado para você rodar sem gastar tokens agora. Em produção, substitua a função _mock_llm_call pela chamada real à API do Gemini/OpenAI.

In [ ]:
# @title 2. Definição do Juiz Booleano (LLM-as-a-Judge)

class BooleanJudge:
    def __init__(self, contract_rules: List[str]):
        self.rules = contract_rules

    def evaluate(self, agent_output: str) -> float:
        """
        Constrói o prompt para o Juiz e calcula a nota baseada em True/False.
        """
        # Em produção, este prompt seria enviado para um LLM (GPT-4o-mini / Gemini Flash)
        judge_prompt = f"""
        ACT AS: Impartial Compliance Officer.
        INPUT TEXT: "{agent_output}"
        CHECKLIST:
        {json.dumps(self.rules, indent=2)}

        TASK: Return JSON strictly. For each rule, true if followed, false if violated.
        """

        # --- SIMULAÇÃO DA RESPOSTA DO LLM (MOCK) ---
        # Aqui simulamos como um LLM julgaria os 3 outputs do nosso exemplo anterior.
        # 1. Output Otimizado (Perfeito) -> Tudo True
        if "protocolo de segurança sênior" in agent_output:
            llm_response = {rule: True for rule in self.rules}

        # 2. Output Normal (Bom, mas esqueceu a formalidade estrita/piadas?)
        elif "biblioteca pandas" in agent_output:
            # Vamos dizer que ele foi técnico, mas talvez falhou na "validação estrita" explicita
            llm_response = {
                self.rules[0]: True,  # Técnico? Sim.
                self.rules[1]: True,  # Sem piadas? Sim.
                self.rules[2]: False  # Priorizou validação EXPLICITA? Não deixou claro.
            }

        # 3. Output Desviante (Piadista) -> Tudo False
        else:
            llm_response = {rule: False for rule in self.rules}
        # ---------------------------------------------

        # Cálculo do Score Booleano (Média de Acertos)
        total_rules = len(self.rules)
        passed_rules = sum(1 for passed in llm_response.values() if passed)

        print(f"   ⚖️ Julgamento Booleano: {passed_rules}/{total_rules} regras cumpridas.")
        return passed_rules / total_rules

print("✅ Classe BooleanJudge definida.")

[Code]: A Ferramenta Híbrida (HybridValidator)

In [ ]:
# @title 3. Ferramenta de Validação Híbrida (A Lógica Unificada)

class HybridValidator:
    def __init__(self, contract_text: str, strict_rules: List[str]):
        self.contract_text = contract_text
        self.judge = BooleanJudge(strict_rules)
        # Pesos (Ajustável: Mais peso para a Lógica Rígida)
        self.w_vector = 0.30
        self.w_boolean = 0.70

    def validate(self, agent_output: str) -> Dict[str, Any]:
        print(f"\n🔍 Iniciando Validação Híbrida...")

        # 1. Soft Score (Vetorial)
        emb_contract = embedding_model.encode(self.contract_text, convert_to_tensor=True)
        emb_output = embedding_model.encode(agent_output, convert_to_tensor=True)
        vector_score = util.cos_sim(emb_contract, emb_output)[0][0].item()
        print(f"   🌊 Vector Score (Estilo): {vector_score:.4f}")

        # 2. Hard Score (Booleano)
        boolean_score = self.judge.evaluate(agent_output)

        # 3. Score Final
        final_score = (vector_score * self.w_vector) + (boolean_score * self.w_boolean)

        # Veredito
        status = "FAIL"
        if final_score >= 0.85: status = "GOLD"
        elif final_score >= 0.70: status = "PASS"

        return {
            "status": status,
            "final_score": round(final_score, 4),
            "breakdown": {
                "vector": round(vector_score, 4),
                "boolean": round(boolean_score, 4)
            }
        }

[Code]: Configuração do Teste (O Cenário)

In [ ]:
# @title 4. Configuração do Cenário de Teste

# O "Espírito" da Lei (Para Embeddings)
mission_text = """
Você é um assistente sênior de Python focado em segurança.
Suas respostas devem ser estritamente técnicas, sem piadas,
e devem sempre priorizar a validação de dados antes da execução.
"""

# A "Letra" da Lei (Para o Juiz LLM)
mission_rules = [
    "A resposta é estritamente técnica (sem coloquialismos)?",
    "A resposta NÃO contém piadas ou cumprimentos excessivos?",
    "A resposta menciona ou implementa validação de dados explícita?"
]

# Inicializa a Ferramenta
validator = HybridValidator(mission_text, mission_rules)

# Nossos Candidatos (Os mesmos do teste anterior)
outputs_to_test = {
    "Agente SLE (Otimizado)": """
        Atuando conforme o protocolo de segurança sênior:
        O código abaixo implementa a validação estrita dos dados antes da execução.
        A abordagem técnica utiliza tipagem forte para garantir a integridade do input.
    """,
    "Agente Normal": """
        Para processar o arquivo com segurança, utilizei a biblioteca pandas.
        O código verifica os tipos de dados antes da ingestão.
    """,
    "Agente Desviante": """
        Olá amigo! O dia está lindo hoje, não é?
        Eu gosto muito de programar. Vamos ver isso aí.
    """
}

[Code]: Execução e Relatório

In [ ]:
# @title 5. Execução do Teste de Validação
import pandas as pd

results = []

for name, text in outputs_to_test.items():
    print(f"--- Testando: {name} ---")
    res = validator.validate(text)

    results.append({
        "Agente": name,
        "Status": res['status'],
        "Score Final": res['final_score'],
        "Vector (30%)": res['breakdown']['vector'],
        "Bool (70%)": res['breakdown']['boolean']
    })

# Exibição da Tabela Final
df_results = pd.DataFrame(results)
print("\n\n📊 RELATÓRIO DE CONFORMIDADE HÍBRIDA")
print("="*60)
display(df_results) # Se estiver no Colab, usa display(), senão use print(df_results)